# Identify best ML model by fitting various parameters

In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import PrecisionRecallDisplay, RocCurveDisplay
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import make_pipeline
from collections import Counter
## my functions/classes
from mb_xai import mb_utils
import mb_xai.gut_data as gd
from sklearn.decomposition import PCA
from sklearn.feature_selection import f_classif
# import mb_xai.mb_data as gd
# import mb_xai.mb_learn as gl

In [2]:
def filter_X_cols(X_df, std_thresh=1e-3, verbose=False):
    """Drop features in X_df that are all 0, or the same number (or have very little std)"""
    if verbose==True:
        print(X_df.shape)
    X_df = X_df[X_df.columns[X_df.std()>std_thresh]]
    if verbose==True:
        print(X_df.shape)
    return X_df

def match_Xy_df(X_in_df, y_in_df):
    X_df, y_df = X_in_df.copy(), y_in_df.copy()
    """ Makes sure X_df and y_df have the same indices"""
    overlap = list(set(X_df.index).intersection(set(y_df.index)))
    X_df =  X_df.loc[overlap]
    y_df =  y_df.loc[overlap]
    return X_df, y_df

In [3]:
FIG_SAVE_LOC = "../figures/"
# DATA_LOC = '../../../Data/community_optimization/data/'
DATA_LOC = '../../../Data/microbiome_xai/'

gut_data = gd.GutData()
gut_data.load_data(
    # FILE_COMM_MODEL='../data/reconstructions/community_5_TOP-vegan.pickle',
    # FILE_COMM_MODEL= DATA_LOC + 'reconstructions/community_5_TOP-vegan.pickle',
    FILE_COMM_MODEL= DATA_LOC + 'reconstructions/community_50_TOP.pickle',
    # FILE_GENUS_ASVS = "../data/agp_data/taxon_genus_asvs.csv",
    FILE_GENUS_ASVS = DATA_LOC + 'agp_data/taxon_genus_asvs.csv',
    # FILE_METADATA = DATA_LOC + "agp_data/mcdonald_agp_metadata.txt",
    FILE_METADATA = DATA_LOC + "agp_data/metadata_biosample_filtered.csv",
    DIR_SIM_DATA = DATA_LOC + "micom-sim-data/"  # "../data/micom-sim-data/",
)
gut_data.norm_abundances(filter_model=True, add_delta=True) ## Filters genus to those in model, adds small value to abundaces
gut_data.X_df = gut_data.asv_df.T.copy()
gut_data.sample_list = gut_data.X_df.index.to_list()
# gut_data.set_ibs_df(sample_num=20, add_other_diagnosis=False)
gut_data.set_vegan_df(sample_num=200)
## Set vegan df changes X_df and y_df and will therefore change medium_df. Be sure to run medium df after setting samples
medium_df = pd.DataFrame(1000, columns=gut_data.com_model.medium.keys(), index = gut_data.X_df.index)
gut_data.sample_medium_dict = medium_df.T.to_dict()
gut_data.return_fluxes = True
gut_data.pfba_bool = True # otherwise optimum values will not be fluxes but intead min(sum flux)

Fixed EX_tDHNACOA(e)
... normalizing raw ASV abundances to fractions, dropping samples with 0 total abundances
# of genus in model and not filterd out of QA/QC: 50, # of genus not in model:0


In [25]:
# for x in gut_data.metadata_df.columns:
#     if "diabetes" in x or "T2D" in x:
#         print(x)

for x in gut_data.metadata_df.columns:
    if "ibd" in x or "IBD" in x:
        print(x)
        print(Counter(gut_data.metadata_df[x]))

gut_data.metadata_df["ibd"].unique()

ibd
Counter({'I do not have this condition': 734, 'Not provided': 74, 'Diagnosed by a medical professional (doctor, physician assistant)': 4, 'Unknown': 2, 'Self-diagnosed': 1, nan: 1})
ibd_diagnosis
Counter({'Not provided': 757, nan: 55, 'Unspecified': 4})
ibd_diagnosis_refined
Counter({'Not provided': 757, nan: 55, 'Unspecified': 4})
subset_ibd
Counter({'True': 687, 'False': 80, nan: 49})
ibd diagnosis
Counter({nan: 795, 'Not provided': 21})
subset ibd
Counter({nan: 767, 'True': 47, 'False': 2})
ibd diagnosis refined
Counter({nan: 795, 'Not provided': 21})


In [21]:
col_type = "diabetes"
y_df = mb_utils.numeric_metadata_col(gut_data.metadata_df, col_type, encode_type="one-hot")
y_df = mb_utils.drop_notprovided(y_df)

In [22]:
def set_t2d_df(self, add_other_diagnosis=True, sample_num=5):
    """ Helper function for specifying a list of samples that include IBS
    """
    col_type = "diabetes"
    y_df = mb_utils.numeric_metadata_col(self.metadata_df, col_type, encode_type="one-hot")
    y_df = mb_utils.drop_notprovided(y_df)

    ibs_samples = y_df["diabetes_diagnosed by a medical professional (doctor, physician assistant)"][y_df["diabetes_diagnosed by a medical professional (doctor, physician assistant)"]!=0].index.tolist()
    if add_other_diagnosis==True:
        ibs_samples_self = y_df["diabetes_self-diagnosed"][y_df["diabetes_self-diagnosed"]==1].index.tolist()
        # ibs_samples_alt = y_df["ibs_diagnosed by an alternative medicine practitioner"][y_df["ibs_diagnosed by an alternative medicine practitioner"]!=0].index.tolist()
        ibs_samples.extend(ibs_samples_self)
        # ibs_samples.extend(ibs_samples_alt)
    non_ibs_samples = y_df["diabetes_i do not have this condition"][y_df["diabetes_i do not have this condition"]==1].index.tolist()

    # sample_list = vegan_samples+non_vegan_samples[:len(vegan_samples)]
    sample_list = ibs_samples[:sample_num]+non_ibs_samples[:sample_num]
    y_df = y_df["diabetes_diagnosed by a medical professional (doctor, physician assistant)"]
    y_df.loc[ibs_samples]=1
    y_df.loc[non_ibs_samples]=0
    y_df = y_df.loc[sample_list]
    self.y_df = y_df
    self.sample_list = sample_list
    self.X_df = self.asv_df.T #.loc[sample_list]
    ### match the indices of X and y
    self.match_Xy_df()

,"diabetes_diagnosed by a medical professional (doctor, physician assistant)",diabetes_i do not have this condition,diabetes_self-diagnosed,diabetes_unknown
sample_name,,,,
10317.000001886,0,1,0,0
10317.000001078,0,1,0,0
10317.000002229,0,1,0,0
10317.000001026,0,1,0,0
10317.000002665,0,1,0,0
...,...,...,...,...
10317.000001161,0,1,0,0
10317.000002224,0,1,0,0
10317.000001281,0,1,0,0


In [ ]:
# FLUX_DF_NAME = "micom_fluxes-top5-109_samples.csv"
FLUX_DF_NAME = "micom_medium-fluxes-top5-736_samples.csv"
X_flux = pd.read_csv(gut_data.dir_sim_data+FLUX_DF_NAME,index_col=0)
X_flux.index = X_flux.index.astype(str)

In [ ]:
##### -------- Iterate over different ML params ---------
## FLUX_DF_NAME = "micom_fluxes-top5-109_samples.csv"
FLUX_DF_NAME = "micom_medium-fluxes-top5-736_samples.csv"
X_flux = pd.read_csv(gut_data.dir_sim_data+FLUX_DF_NAME,index_col=0)
X_flux.index = X_flux.index.astype(str)

# RFE test
n_splits = 20 #10
test_size = 0.25
# input_type = "vegan"
input_type = "ibs"

y_df = gut_data.y_df.copy()
X, y = match_Xy_df(X_flux, y_df)
# X, y = match_Xy_df(gut_data.X_df, y_df)
# X = mb_utils.filter_X_cols(X)
X, y = X.values, y.values

score_list = ['accuracy','balanced_accuracy','roc_auc','average_precision']

param_score_dict = {}
for c_param in [1e-3, 1e-2, 1e-1, 1, 5, 1e1, 1e2, 1e3]:
    for n_features in [5, 10, 15, 20, 35, 50]:
        for penalty_param in ["l1", "l2"]:

            gut_data.logreg.C = c_param
            gut_data.logreg.penalty = penalty_param

            # feat_filter = RFE(estimator=gut_data.logreg, n_features_to_select=n_features)
            feat_filter = SelectKBest(f_classif, k=n_features)

            skf = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size)
            model = make_pipeline(StandardScaler(), SMOTE(), feat_filter, gut_data.logreg)
            # model = make_pipeline(StandardScaler(), RandomOverSampler(), feat_filter, gut_data.logreg)
            n_scores = cross_validate(model, X, y, scoring=score_list, cv=skf, n_jobs=-1, error_score='raise')
            score_dict = {score_id: n_scores["test_"+score_id].mean() for score_id in score_list}
            param_score_dict.update({(c_param, n_features, penalty_param): score_dict})

score_df = pd.DataFrame(param_score_dict).T

In [ ]:
N_SPLITS = 10 #10
TEST_SIZE = 0.25
SAMPLE_NUM = 200

for phenotype in ["vegan", "ibs", "t2db"][:]:
    gut_data = gd.GutData()
    gut_data.load_data(
        # FILE_COMM_MODEL= DATA_LOC + 'reconstructions/community_5_TOP-vegan.pickle',
        FILE_COMM_MODEL= DATA_LOC + 'reconstructions/community_50_TOP.pickle',
        FILE_GENUS_ASVS = DATA_LOC + 'agp_data/taxon_genus_asvs.csv',
        FILE_METADATA = DATA_LOC + "agp_data/metadata_biosample_filtered.csv",
        DIR_SIM_DATA = DATA_LOC + "micom-sim-data/"
    )
    gut_data.norm_abundances(filter_model=True, add_delta=True) ## Filters genus to those in model, adds small value to abundaces
    gut_data.X_df = gut_data.asv_df.T.copy()
    gut_data.sample_list = gut_data.X_df.index.to_list()

    if phenotype=="vegan":
        gut_data.set_vegan_df(sample_num=SAMPLE_NUM)
    elif phenotype=="ibs":
        gut_data.set_ibs_df(sample_num=SAMPLE_NUM, add_other_diagnosis=False)
    elif phenotype=="t2db":
        gut_data.set_t2d_df(sample_num=SAMPLE_NUM, add_other_diagnosis=False)


    y_df = gut_data.y_df.copy()
    X, y = match_Xy_df(X_flux, y_df)
    # X, y = match_Xy_df(gut_data.X_df, y_df)
    # X = mb_utils.filter_X_cols(X)
    X, y = X.values, y.values

    score_list = ['accuracy','balanced_accuracy','roc_auc','average_precision']

    param_score_dict = {}
    for c_param in [1e-3, 1e-2, 1e-1, 1, 5, 1e1, 1e2, 1e3]:
        for n_features in [5, 10, 15, 20, 35, 50]:
            for penalty_param in ["l1", "l2"]:

                gut_data.logreg.C = c_param
                gut_data.logreg.penalty = penalty_param

                # feat_filter = RFE(estimator=gut_data.logreg, n_features_to_select=n_features)
                feat_filter = SelectKBest(f_classif, k=n_features)

                skf = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size)
                model = make_pipeline(StandardScaler(), SMOTE(), feat_filter, gut_data.logreg)
                # model = make_pipeline(StandardScaler(), RandomOverSampler(), feat_filter, gut_data.logreg)
                n_scores = cross_validate(model, X, y, scoring=score_list, cv=skf, n_jobs=-1, error_score='raise')
                score_dict = {score_id: n_scores["test_"+score_id].mean() for score_id in score_list}
                param_score_dict.update({(c_param, n_features, penalty_param): score_dict})

    score_df = pd.DataFrame(param_score_dict).T
    score_df.to_csv(gut_data.dir_sim_data+"ml_params_"+phenotype)